#**Installations**

In [ ]:
!pip install prettytable

In [ ]:
import string
from prettytable import PrettyTable
import time
import sys
import hashlib
import math
import random
from collections import Counter
from typing import Dict, List, Tuple
from collections import Counter
import math
import string
import random
from prettytable import PrettyTable
import time
import numpy as np
from scipy.stats import entropy as scipy_entropy
from itertools import combinations
from tqdm import tqdm

#**Scoring Class**

In [ ]:
class Scoring:
    """
    The Scoring class is responsible for evaluating the effectiveness of the encryption
    method used in the Cipher class. It applies a series of metrics to the encrypted string
    and assigns a score based on these metrics.
    """

    def __init__(self, cipher: 'Cipher', running_time: float, weights: Dict[str, float], max_length_multiplier: float = 2.0) -> None:
        """
        The constructor for the Scoring class.
        Parameters:
            cipher (Cipher): An instance of the Cipher class.
            running_time (float): The time taken to run the encryption.
            weights (Dict[str, float]): A dictionary containing the weights for each metric.
            max_length_multiplier (float): The maximum allowed ratio of the length of the encrypted string
                                           to the length of the original string.
        """
        self.cipher = cipher
        self.weights = weights
        self.max_length_multiplier = max_length_multiplier
        self.running_time = running_time
        self.frequency = Counter(self.cipher.encrypted_string)
        self.score = self.calculate_score()
        self.summary = self.generate_summary()


    # Define metrics
    def unique_chars_metric(self) -> float:
        """
        Evaluates the number of unique characters in the encrypted string.
        Returns:
            float: The number of unique characters.
        """
        return len(set(self.cipher.encrypted_string)) / len(set(string.printable))

    def distinct_sequences_metric(self) -> float:
        """
        Evaluates the number of distinct character sequences in the encrypted string.
        Returns:
            float: The number of distinct character sequences.
        """
        # Updated to count sequences of two characters
        sequences = [''.join(seq) for seq in zip(self.cipher.encrypted_string, self.cipher.encrypted_string[1:])]
        return len(set(sequences)) / max(1, len(set(combinations(string.printable, 2))))

    def entropy_metric(self) -> float:
        """
        Evaluates the entropy of the encrypted string.
        Returns:
            float: The entropy of the encrypted string.
        """
        # Updated to use scipy's entropy function for better precision
        probabilities = [count / len(self.cipher.encrypted_string) for count in self.frequency.values()]
        return scipy_entropy(probabilities, base=2) / math.log2(len(set(string.printable)))

    def frequency_analysis_metric(self) -> float:
        """
        Evaluates the result of a frequency analysis on the encrypted string.
        Returns:
            float: The result of a frequency analysis on the encrypted string.
        """
        most_common_char_frequency = self.frequency.most_common(1)[0][1] / len(self.cipher.encrypted_string)
        return 1 - most_common_char_frequency

    def length_consistency_metric(self) -> float:
        """
        Evaluates how the length of the string changes after encryption.
        Returns:
            float: The absolute difference between the length of the original and encrypted strings.
        """
        return abs(len(self.cipher.encrypted_string) - len(self.cipher.original_string)) / len(self.cipher.original_string)

    def evenness_metric(self) -> float:
        """
        Evaluates how evenly distributed the characters in the encrypted string are.
        Returns:
            float: The standard deviation of the character frequencies.
        """
        frequencies = list(self.frequency.values())
        mean_freq = sum(frequencies) / len(frequencies)
        variance = sum((freq - mean_freq) ** 2 for freq in frequencies) / len(frequencies)
        return 1 - (math.sqrt(variance) / len(self.cipher.encrypted_string))

    def reversibility_metric(self) -> float:
        """
        Evaluates whether the original string can be correctly retrieved by decryption.
        Returns:
            float: 1.0 if the decrypted string matches the original string, 0.0 otherwise.
        """
        decrypted_string = self.cipher.decrypt()
        return float(decrypted_string == self.cipher.original_string)

    def change_propagation_metric(self) -> float:
        """
        Evaluates how much the encrypted string changes when a small change
        is made to the original string.
        Returns:
            float: The measure of change propagation.
        """
        # Change the original string
        changed_string = 'a' + self.cipher.original_string[1:]

        # Create a new Cipher instance with the changed string and encrypt it
        cipher_changed = Cipher(changed_string)
        cipher_changed.encrypt()
        changed_encrypted_string = cipher_changed.encrypted_string

        # Calculate the Levenshtein distance
        levenshtein_distance = self._levenshtein_distance(self.cipher.encrypted_string, changed_encrypted_string)

        return levenshtein_distance / len(self.cipher.encrypted_string)

    def pattern_analysis_metric(self) -> float:
        """
        Evaluates how patterns from the original string are unrecognizable in the encrypted string.
        Returns:
            float: The measure of pattern preservation.
        """
        pattern_instances_orig = sum(1 for a, b in zip(self.cipher.original_string, self.cipher.original_string[1:]) if a == b)
        pattern_instances_enc = sum(1 for a, b in zip(self.cipher.encrypted_string, self.cipher.encrypted_string[1:]) if a == b)

        if pattern_instances_orig == 0:
            return 1.0  # No repeated patterns in the original string
        elif pattern_instances_enc == 0:
            return 1.0  # No repeated patterns in the encrypted string
        else:
            return 1 - (pattern_instances_enc / pattern_instances_orig)


    def correlation_analysis_metric(self) -> float:
        """
        Calculates the correlation of the character at each position with the character
        at the same position in the encrypted string.
        Returns:
            float: The average correlation for all positions.
        """
        correlations = sum(1 for a, b in zip(self.cipher.original_string, self.cipher.encrypted_string) if a == b)
        correlation_metric = correlations / len(self.cipher.original_string)
        return 1 - correlation_metric

    def complexity_metric(self) -> float:
        """
        Evaluates the complexity of the encryption by measuring the length of the encrypted string.
        Returns:
            float: The length of the encrypted string divided by the length of the original string.
        """
        return len(self.cipher.encrypted_string) / len(self.cipher.original_string)

    def randomness_metric(self) -> float:
        """
        Evaluates how random the encrypted string appears.
        Returns:
            float: The standard deviation of the character frequencies in the encrypted string of a random string.
        """
        random_string = ''.join(random.choice(string.printable) for _ in range(len(self.cipher.original_string)))
        cipher = Cipher(random_string)
        cipher.encrypt()
        random_encrypted_string = cipher.encrypted_string
        random_encrypted_frequency = Counter(random_encrypted_string)
        frequencies = list(random_encrypted_frequency.values())
        mean_freq = sum(frequencies) / len(frequencies)
        variance = sum((freq - mean_freq) ** 2 for freq in frequencies) / len(frequencies)
        return math.sqrt(variance) / len(string.printable)

    def normalized_levenshtein_metric(self) -> float:
        """
        Evaluates the normalized Levenshtein distance between the original string and the decrypted string.
        Returns:
            float: The normalized Levenshtein distance.
        """
        decrypted_string = self.cipher.decrypt()
        distance = self._levenshtein_distance(self.cipher.original_string, decrypted_string)
        return 1 - distance / max(len(self.cipher.original_string), len(decrypted_string))

    def encryption_consistency_metric(self) -> float:
        """
        Evaluates the consistency of the encryption method.
        Returns:
            float: The consistency of the encryption method.
        """
        original_string = self.cipher.original_string
        original_encrypted_string = self.cipher.encrypted_string

        self.cipher.original_string = original_string
        self.cipher.encrypted_string = ""
        self.cipher.encrypt()
        second_encryption = self.cipher.encrypted_string

        # Reset the encrypted_string back to its original value
        self.cipher.encrypted_string = original_encrypted_string

        return float(original_encrypted_string == second_encryption)

    def running_time_metric(self) -> float:
        """
        Evaluates the time taken for the encryption process.
        Returns:
            float: The time taken for the encryption process.
        """
        # Use numpy's clip function to ensure the returned value is between 0 and 1
        return np.clip(1 - self.running_time, 0, 1)  # Updated to use the new attribute


    def calculate_score(self) -> float:
        """
        Calculates the total score based on the defined metrics.

        Returns:
            float: The total score.
        """
        total_score = 0
        try:
            if len(self.cipher.encrypted_string) > len(self.cipher.original_string) * self.max_length_multiplier:
                raise Exception("The length of the encrypted string exceeds the allowed limit. This entry is disqualified.")

            total_score += self.weights["unique_chars"] * self.unique_chars_metric()
            total_score += self.weights["distinct_sequences"] * self.distinct_sequences_metric()
            total_score += self.weights["entropy"] * self.entropy_metric()
            total_score += self.weights["frequency_analysis"] * self.frequency_analysis_metric()
            total_score += self.weights["length_consistency"] * self.length_consistency_metric()
            total_score += self.weights["evenness"] * self.evenness_metric()
            total_score += self.weights["reversibility"] * self.reversibility_metric()
            total_score += self.weights["change_propagation"] * self.change_propagation_metric()
            total_score += self.weights["pattern_analysis"] * self.pattern_analysis_metric()
            total_score += self.weights["correlation_analysis"] * self.correlation_analysis_metric()
            total_score += self.weights["complexity"] * self.complexity_metric()
            total_score += self.weights["randomness"] * self.randomness_metric()
            total_score += self.weights["normalized_levenshtein"] * self.normalized_levenshtein_metric()
            total_score += self.weights["encryption_consistency"] * self.encryption_consistency_metric()
            total_score += self.weights["running_time"] * self.running_time_metric()
        except Exception as e:
            print(f"An error occurred while calculating the score: {e}")
            return 0

        return total_score

    def _levenshtein_distance(self, s1: str, s2: str) -> int:
        """
        Calculates the Levenshtein distance between two strings.

        Parameters:
            s1 (str): The first string.
            s2 (str): The second string.

        Returns:
            int: The Levenshtein distance between the two strings.
        """
        if len(s1) < len(s2):
            return self._levenshtein_distance(s2, s1)

        # len(s1) >= len(s2)
        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]


    def generate_summary(self) -> Dict[str, float]:
        """
        Generates a summary of the metrics used to calculate the score.
        Returns:
            Dict[str, float]: A dictionary containing the metric names and their values.
        """
        summary = {
            "unique_chars": self.unique_chars_metric(),
            "distinct_sequences": self.distinct_sequences_metric(),
            "entropy": self.entropy_metric(),
            "frequency_analysis": self.frequency_analysis_metric(),
            "length_consistency": self.length_consistency_metric(),
            "evenness": self.evenness_metric(),
            "reversibility": self.reversibility_metric(),
            "change_propagation": self.change_propagation_metric(),
            "pattern_analysis": self.pattern_analysis_metric(),
            "correlation_analysis": self.correlation_analysis_metric(),
            "complexity": self.complexity_metric(),
            "randomness": self.randomness_metric(),
            "normalized_levenshtein": self.normalized_levenshtein_metric(),
            "encryption_consistency": self.encryption_consistency_metric(),
            "running_time": self.running_time_metric()  # New metric
        }
        return summary

    @staticmethod
    def print_results(table_data: List[List[str]], scores: List[float]) -> None:
        """
        Prints the results of the encryption tests in a table format.

        Parameters:
            table_data (list): A list of lists where each sublist contains the test number, original string,
                              encrypted string, decrypted string, decryption success, score, and running time.
            scores (list): A list of all scores.
        """
        try:
            # Create a PrettyTable instance
            table = PrettyTable()

            # Set the column names
            table.field_names = ["Test No.", "Original String", "Encrypted String",
                                "Decrypted String", "Decryption Success", "Score", "Running Time"]

            # Add each test's results to the table
            for data in table_data:
                table.add_row(data)

            # Print the table
            print(table)

            # Print the average score
            print(f"\nAverage score: {sum(scores) / len(scores)}")
        except Exception as e:
            print(f"An error occurred while printing the results: {e}")


#**Integer Encoding Function**

In [ ]:
def generate_custom_charset():
    # Defining unicode_ranges with the desired character ranges
    unicode_ranges = [
    (0x0020, 0x007F), (0x2580, 0x259F),(0x00A0, 0x00FF), (0x25A0, 0x25FF),(0x0100, 0x017F), (0x2600, 0x26FF),(0x0180, 0x024F), (0x2700, 0x27BF),(0x0250, 0x02AF), (0x27C0, 0x27EF),
    (0x02B0, 0x02FF), (0x27F0, 0x27FF),(0x0300, 0x036F), (0x2800, 0x28FF),(0x0370, 0x03FF), (0x2900, 0x297F),(0x0400, 0x04FF), (0x2980, 0x29FF),(0x0500, 0x052F), (0x2A00, 0x2AFF),(0x0530, 0x058F), (0x2B00, 0x2BFF),
    (0x0590, 0x05FF), (0x2E80, 0x2EFF),(0x0600, 0x06FF), (0x2F00, 0x2FDF),(0x0700, 0x074F), (0x2FF0, 0x2FFF),(0x0780, 0x07BF), (0x3000, 0x303F),
    (0x0900, 0x097F), (0x3040, 0x309F),(0x0980, 0x09FF), (0x30A0, 0x30FF),(0x0A00, 0x0A7F), (0x3100, 0x312F),(0x0A80, 0x0AFF), (0x3130, 0x318F),
    (0x0B00, 0x0B7F), (0x3190, 0x319F),(0x0B80, 0x0BFF), (0x31A0, 0x31BF),
    (0x0C00, 0x0C7F), (0x31F0, 0x31FF),(0x0C80, 0x0CFF), (0x3200, 0x32FF),(0x0D00, 0x0D7F), (0x3300, 0x33FF),(0x0D80, 0x0DFF),
    (0x0E00, 0x0E7F), (0x4DC0, 0x4DFF),(0x0E80, 0x0EFF),(0x0F00, 0x0FFF), (0xA000, 0xA48F),(0x1000, 0x109F), (0xA490, 0xA4CF),
    (0x10A0, 0x10FF), (0x1100, 0x11FF),(0x1200, 0x137F),(0x13A0, 0x13FF),(0x1400, 0x167F),(0x1680, 0x169F),
    (0x16A0, 0x16FF), (0xFB00, 0xFB4F),(0x1700, 0x171F),(0x1720, 0x173F),(0x1740, 0x175F), (0xFE20, 0xFE2F),(0x1760, 0x177F), (0xFE30, 0xFE4F),
    (0x1780, 0x17FF), (0xFE50, 0xFE6F),(0x1800, 0x18AF), (0xFE70, 0xFEFF),(0x1900, 0x194F), (0xFF00, 0xFFEF),(0x1950, 0x197F), (0xFFF0, 0xFFFF),(0x19E0, 0x19FF), (0x10000, 0x1007F),(0x1D00, 0x1D7F), (0x10080, 0x100FF),
    (0x1E00, 0x1EFF), (0x10100, 0x1013F),(0x1F00, 0x1FFF), (0x10300, 0x1032F),(0x2000, 0x206F), (0x10330, 0x1034F),(0x2070, 0x209F), (0x10380, 0x1039F),(0x20A0, 0x20CF), (0x10400, 0x1044F),(0x20D0, 0x20FF), (0x10450, 0x1047F),
    (0x2100, 0x214F), (0x10480, 0x104AF),(0x2150, 0x218F), (0x10800, 0x1083F),
    (0x2190, 0x21FF), (0x1D000, 0x1D0FF),(0x2200, 0x22FF), (0x1D100, 0x1D1FF),(0x2300, 0x23FF), (0x1D300, 0x1D35F),(0x2400, 0x243F), (0x1D400, 0x1D7FF),(0x2440, 0x245F),(0x2460, 0x24FF),(0x2500, 0x257F)
    ]

    # Shuffle the unicode_ranges to randomize the order
    random.shuffle(unicode_ranges)

    all_characters = []

    for start, end in unicode_ranges:
        # Generate characters for the current range
        characters = [chr(code) for code in range(start, end + 1)]
        all_characters.extend(characters)

    # Combine characters into a string
    custom_charset_unrandomized = ''.join(all_characters)

    # Encode the characters as bytes using utf-8
    custom_charset_bytes = custom_charset_unrandomized.encode('utf-8', 'replace')

    # Decode the bytes back into a string
    custom_charset_first = custom_charset_bytes.decode('utf-8', 'replace')

    # Convert the string into a list and shuffle the list
    custom_charset_list = list(custom_charset_first)
    random.shuffle(custom_charset_list)

    # Convert the shuffled list back into a string
    custom_charset = ''.join(custom_charset_list)

    return custom_charset

def encode_large_integer(number, charset):
    base = len(charset)
    encoded_chars = []

    while number > 0:
        remainder = number % base # Calculate the remainder when dividing the number by the base.
        encoded_chars.append(charset[remainder]) # Append the character at the remainder's index in the charset.
        number //= base  # Update the number by dividing it by the base (integer division).

    #Reverse the order of encoded characters and join them to create the final encoded string.
    #Reversing is necessary because we append remainders from least significant to most significant.
    return ''.join(encoded_chars[::-1])


def decode_large_integer(encoded_str, charset, separatedInts):
    base = len(charset) # Determine the base of the decoding from the length of the charset.
    decoded_number = 0 # Initialize the decoded number.


    for char in encoded_str:
        value = charset.index(char) # Find the index of the character in the charset.
        decoded_number = decoded_number * base + value # Update the decoded number using base multiplication.

    splitInts = [str(number) for number in separatedInts]

    encrypted_ascii = []

    bigNumber = ""

    for number in str(decoded_number):
        bigNumber += number
        if bigNumber in splitInts:
            encrypted_ascii.append(bigNumber)
            splitInts.remove(bigNumber)
            bigNumber = ""


    return [int(num) for num in encrypted_ascii]

#Initialize Custom Characterset
custom_charset = generate_custom_charset()

#**Encryption & Decryption Class**

In [ ]:
class RSA_Algo:
    def __init__(self, range1, range2, original_string):
      # Initialize instance variables
        self.original_string = original_string
        self.consistencyVar = ord(original_string[0])
        self.range1 = range1
        self.range2 = range2
        self.prime_a = None
        self.prime_b = None
        self.product = None
        self.phi_n = None
        self.public_key = None
        self.private_key = None
        self.encrypted_str_list = []
        self.ciphertext = ""
        random.seed(self.consistencyVar) # Set seed for the prime generation process to maintain consistency

    def is_prime(self,num): # Check if a number is prime
      if num < 2:
        return False
      for i in range(2, int(num ** 0.5) + 1):
          if num % i == 0:
              return False
      return True

    def generate_prime(self):  # Generate a random prime number within the specified range
      while True:
        num = random.randint(self.range1, self.range2)
        if self.is_prime(num):
            return num

    def generate_key_pair(self):
        while True: # Generate two distinct prime numbers
            self.prime_a = self.generate_prime()
            self.prime_b = self.generate_prime()
            if self.prime_a != self.prime_b:
                break

        self.product = self.prime_a * self.prime_b # Compute the product of the two prime numbers
        self.phi_n = (self.prime_a - 1) * (self.prime_b - 1)  # Calculate Euler's totient function

        # Generate the public key
        pub_randA = random.randint(512,2048)
        pub_randB = random.randint(4096,8192)
        for i in range(pub_randA,pub_randB):
            if math.gcd(self.phi_n, i) == 1: # Check if i and phi_n are coprime
                self.public_key = i
                break

        public_key = (self.public_key, self.product)

        # Private Key
        i = 1
        while True:
            private = ((self.phi_n * i) + 1) / self.public_key # Calculate potential private key
            i += 1
            if private % self.public_key == 0: # Check if private key satisfies the modulo inverse condition
                self.private_key = int(private)
                break

        private_key = (self.private_key, self.product)

        return public_key, private_key

    def encrypt(self):

        plaintext = self.original_string[::-1] # Reverse the plaintext for added security

        while True:

          self.generate_key_pair() # Initiate Key Generation Process

          ascii_list = [] # Intialize empty list to store ascii values of corresponding plaintext characters

          for char in plaintext:
              ascii_list.append(ord(char))

          encrypted_ascii = [] # Intialize empty list for storing RSA encrypted values

          for ascii_values in ascii_list:
              encrypted = pow(ascii_values, self.public_key, self.product) # Encrypted using our generated Public Key and Product of Intial prime_a and prime_b values choosen
              encrypted_ascii.append(encrypted)

          self.encrypted_str_list = encrypted_ascii

          self.big_int = int("".join(str(x) for x in encrypted_ascii)) # Concat all the encrypted values into 1 single big Integer

          encoded_ciphertext = encode_large_integer(self.big_int, custom_charset) # Encode the Integer to smaller string using vast unicode characterset

          if len(encoded_ciphertext) <= len(plaintext) * 2: # Make sure the ciphertext follows our condition of cipher <= 2*plaintext
              break

        self.ciphertext = encoded_ciphertext

    def decrypt(self):

        encrypted_ascci_decoded = decode_large_integer(self.ciphertext, custom_charset, self.encrypted_str_list) # Retrieve our encrypted ASCII list by decoding the encoded ciphertext

        decrypted_table = []

        # Finally run the decryption process and generate each original ASCII values from their cipher integer got from RSA
        for encrypted_char in encrypted_ascci_decoded:
            decrypted = pow(encrypted_char, self.private_key, self.product)
            decrypted_table.append(decrypted)

        decrypted_text = ""

        # Convert the decrypted ASCII values into original string characters

        for asciiValues in decrypted_table:
            decrypted_text += chr(asciiValues)

        return decrypted_text[::-1]

In [ ]:
class Cipher:

    def __init__(self, original_string: str) -> None:

        self.original_string = original_string
        self.encrypted_string = ""

    def encrypt(self) -> None:
      # Determines range from where to pick prime number, the higher the range the better the encryption
        self.range1 = 10000
        self.range2 = 15000

        self.rsaobj = RSA_Algo(self.range1,self.range2, self.original_string)

        self.rsaobj.encrypt()

        self.encrypted_string = self.rsaobj.ciphertext

    def decrypt(self) -> str:

        return self.rsaobj.decrypt()

#**Main**

In [ ]:
def main(max_length_multiplier: float):
    """
    The main function that tests the Cipher and Scoring classes.

    Parameters:
        max_length_multiplier (float): The maximum allowed ratio of the length of the encrypted string
                                       to the length of the original string.
    """
    # Ensure the maximum length multiplier is a positive number
    assert max_length_multiplier > 0, "Maximum length multiplier must be a positive number"

    # Define the sample strings
    sample_strings = [
        "Hello, World!",
        "This is a sample string",
        "Another string for testing",
        "A very very long string that should result in a high score",
        "Short string",
        "abcdefghijklmnopqrstuvwxyz",
        "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
        "1234567890",
        "A string with special characters: !@#$%^&*()",
        "A string with spaces    between     words",
        "A string with a mix of letters, numbers, and special characters: abc123!@#"
    ]


    # Define the weights for each metric
    weights = {
        "unique_chars": 0.1,
        "distinct_sequences": 1.0,
        "entropy": 1.5,
        "frequency_analysis": 0.1,
        "length_consistency": 0.5,
        "evenness": 1.2,
        "reversibility": 2.0,
        "change_propagation": 1.5,
        "pattern_analysis": 1.5,
        "correlation_analysis": 1.5,
        "complexity": 1.0,
        "randomness": 1.5,
        "normalized_levenshtein": 1.0,
        "encryption_consistency": 2.0,  # High weight as this is critical
        "running_time": 0.5
    }


    print("\n********** Cipher Testing **********")
    scores = []
    table_data = []

    for i, original_string in enumerate(tqdm(sample_strings, desc= "Encryption In Progress", unit="string"), start=1):
        # Create a Cipher instance and encrypt the string
        cipher = Cipher(original_string)

        # Start the timer
        start_time = time.time()

        cipher.encrypt()

        # End the timer
        end_time = time.time()

        # Calculate the running time
        running_time = end_time - start_time

        # Create a Scoring instance and calculate the score
        scoring = Scoring(cipher, running_time, weights, max_length_multiplier)
        scores.append(scoring.score)

        # Decrypt the string and verify it's the same as the original
        decrypted_string = cipher.decrypt()
        decryption_success = decrypted_string == original_string

        # Add the results to the table data
        table_data.append([
            i, original_string, cipher.encrypted_string,
            decrypted_string, decryption_success, scoring.score, running_time
        ])

    Scoring.print_results(table_data, scores)
    print("\n********** Testing Complete **********\n")

if __name__ == "__main__":

    main(2.0)  # Maximum length multiplier is set to twice the length of the original string


********** Cipher Testing **********


Encryption In Progress: 100%|██████████| 11/11 [00:10<00:00,  1.04string/s]

+----------+----------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------+--------------------+-----------------------+
| Test No. |                              Original String                               |                                                                        Encrypted String                                                                        |                              Decrypted String                              | Decryption Success |       Score        |      Running Time     |
+----------+----------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------